<a href="https://colab.research.google.com/github/robertokamakura/Alura1/blob/main/alura2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai

In [3]:
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)


In [4]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [5]:
title = 'A proxima geracao de IA para desenvolvedores e Google Workspace'
sample_text = ('Titulo: A proxima geracao de IA para desenvolvedores e Google Workspace'
'\n'
'Artigo completo:\n'
'\n'
'Gemini API & Google AI Studio: Uma maneira acessivel de xplorar e criar prototipos com aplicacoes de IA generativa')

embeddings = genai.embed_content(model='models/embedding-001',
                                 content=sample_text,
                                 title=title,
                                 task_type='RETRIEVAL_DOCUMENT')
print(embeddings)

{'embedding': [0.053214602, -0.050558742, -0.019442916, 0.02206584, 0.061121013, -0.0086672595, -0.019351231, -0.033121843, 0.05234061, 0.056943264, 0.016890638, 0.011605252, -0.04973929, -0.021320732, 0.00018200903, -0.0134326, 0.014283665, -0.010237701, -0.037511665, -0.004225399, 0.00904634, 0.0075820144, -0.031248588, -0.07825237, -0.015560413, 0.01929522, 0.02183098, -0.033609774, -0.04910555, 0.038895257, -0.050188765, 0.04792297, -0.0413632, 0.011926712, -0.023390317, -0.04698981, -0.013911722, -0.043987837, 0.005021079, -0.0017888778, 0.002814518, -0.09000795, -0.02141977, 0.010398907, -0.004813, -0.025260605, 0.038929235, 0.048322104, 0.019659402, -0.05573121, 0.025262885, 0.016073765, 0.07083949, -0.041088823, 0.005420264, -0.021568913, 0.029580075, -0.035843436, 0.025838958, 0.007646367, -0.013193462, 0.026331374, -0.009139946, 0.06096115, 0.029420154, -0.06323543, -0.04270298, 0.0014876379, 0.007086045, 0.03186677, 0.00037762336, 0.0006836676, 0.057064194, -0.064371645, -0.

In [7]:
DOCUMENT1 = {
    'Titulo': 'Operacao do sistema de controle cimatico',
    'Conteudo': 'O Googlecar tem um sistema de controle climatico que permite ajustar a temperatura e o fluxo de ar no carro'}
DOCUMENT2 = {
    'Titulo': 'Touchscreen',
    'Conteudo': 'O seu Googlecar tem uma grande tela sensivel ao toque que fornece acess a uma variedade de recursos, incluindo navegacao'}
DOCUMENT3 = {
    'Titulo': 'Mudanca de marchas',
    'Conteudo': 'Seu Googlecar tem uma transmissao automática. Para trocar as marchas, basta mover a alavanca de cambio para a posicao desejada'}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [8]:
df = pd.DataFrame(documents)


,Titulo,Conteudo
0,Operacao do sistema de controle cimatico,O Googlecar tem um sistema de controle climati...
1,Touchscreen,O seu Googlecar tem uma grande tela sensivel a...
2,Mudanca de marchas,Seu Googlecar tem uma transmissao automática. ...


In [14]:
model = 'models/embedding-001'

In [18]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                             content=text,
                             title=title,
                             task_type='RETRIEVAL_DOCUMENT')['embedding']

In [19]:
df['Embeddings'] = df.apply(lambda row: embed_fn(row['Titulo'], row['Conteudo']), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Operacao do sistema de controle cimatico,O Googlecar tem um sistema de controle climati...,"[-0.01274528, -0.019749427, -0.0633264, 0.0448..."
1,Touchscreen,O seu Googlecar tem uma grande tela sensivel a...,"[0.011713132, -0.049997922, -0.020894332, -0.0..."
2,Mudanca de marchas,Seu Googlecar tem uma transmissao automática. ...,"[-0.00953055, -0.040744048, 0.0017229673, 0.03..."


In [21]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                              content=consulta,
                                              task_type='RETRIEVAL_QUERY')['embedding']
  produtos_escalares = np.dot(np.stack(df['Embeddings']), embedding_da_consulta)
  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]['Conteudo']

In [25]:
consulta = 'como trocar as marchas do carro do Google?'
trecho = gerar_e_buscar_consulta(consulta,df,model)
print(trecho)

Seu Googlecar tem uma transmissao automática. Para trocar as marchas, basta mover a alavanca de cambio para a posicao desejada


In [26]:
prompt = f'Reescreva ees texto de uma forma mais descontraída, sem adicionar informacoes que não façam parte do texto: {trecho}'
model_2 = genai.GenerativeModel('gemini-1.0-pro')
response = model_2.generate_content(prompt)
print(response.text)

Seu Googlecar é automático, então nada de se estressar com marchas. É só dar um toquinho na alavanca e pronto!
